In [6]:
import os
import numpy as np
import SimpleITK as sitk
from PIL import Image
import pydicom
import cv2
import nibabel as nib

In [21]:
base_path = "./data/BE_CT"
print(os.listdir(base_path))

data_paths = []
for path in os.listdir(base_path):
    full_path = os.path.join(base_path, path)
    if not os.path.isfile(full_path) and path.find('_') != -1:
        data_paths.append(full_path)

print(data_paths)

['00014844_20200323131101', '00007509_20180911112923', '00008814_20090409111117', '00007509_20181112143811', '00007891_20211207155609', '00013315', '00006185_20190425151208', '00014844', '00006185_20170614125509', '00008814_20180402141431', '00008814_20060605115453', '00016852', '00007509_20190115235012', '00007509_20200612151640', '00016852_20210909090621', '00010992', '00015955_20210304130215', '00009795_20201104110731', '00011525_20230411103414', '00005674', '00015955_20170310113742', '00008142_20070910125040', '00006185_20210830131408', '00006185_20230116082224', '00015955_20230109133539', '00007509_20220831085100', '00007891', '00008814_20080116120147', '00016852_20191118082545', '00014844_20191013122239', '00008814_20140107092531', '00013315_20070823150636', '00007509_20210722093551', '00006185_20200211130252', '00006185_20210513142452', '00009795_20200818131306', '00015955', '00014844_20171228142549', '00009795_20200708085645', '00014844_20200817002821', '00014844_20121123165135

In [22]:
def read_dicom_from_dir(path):
    return pydicom.dcmread(os.path.join(path, os.listdir(path)[0]))

In [23]:
data = read_dicom_from_dir(data_paths[0])
print(data)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 208
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.840.113619.2.452.3.1678417352.10.1584928170.649.26
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.410.200058.1.1.82.1.1
(0002, 0013) Implementation Version Name         SH: 'OFFIS_DCMTK_361'
(0002, 0016) Source Application Entity Title     AE: 'FCT'
-------------------------------------------------
(0008, 0000) Group Length                        UL: 856
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'SECONDARY', 'AXIAL']
(0008, 0012) Instance Creation Date              DA: '20200323'
(0008, 0013) Instance Creation Time  

In [27]:
data_dict = dict()
data_paths.sort()

for path in data_paths:
    imgs=os.listdir(path)
    imgs.sort()
    img_path_list = []
    for img in imgs:
        img_path_list.append(os.path.join(path, img))
    data_dict[os.path.split(path)[-1]]=img_path_list

In [28]:
data_dict

{'00006185_20080731150531': ['./data/BE_CT/00006185_20080731150531/00006185_2008073115053101.dcm',
  './data/BE_CT/00006185_20080731150531/00006185_2008073115053102.dcm',
  './data/BE_CT/00006185_20080731150531/00006185_2008073115053103.dcm',
  './data/BE_CT/00006185_20080731150531/00006185_2008073115053104.dcm',
  './data/BE_CT/00006185_20080731150531/00006185_2008073115053105.dcm',
  './data/BE_CT/00006185_20080731150531/00006185_2008073115053106.dcm',
  './data/BE_CT/00006185_20080731150531/00006185_2008073115053107.dcm',
  './data/BE_CT/00006185_20080731150531/00006185_2008073115053108.dcm',
  './data/BE_CT/00006185_20080731150531/00006185_2008073115053109.dcm',
  './data/BE_CT/00006185_20080731150531/00006185_2008073115053110.dcm',
  './data/BE_CT/00006185_20080731150531/00006185_2008073115053111.dcm',
  './data/BE_CT/00006185_20080731150531/00006185_2008073115053112.dcm',
  './data/BE_CT/00006185_20080731150531/00006185_2008073115053113.dcm',
  './data/BE_CT/00006185_200807311505

In [87]:
def loadFile(filename):
    ds = sitk.ReadImage(filename)
    img_array = sitk.GetArrayFromImage(ds)
    frame_num, width, height = img_array.shape
    return img_array, frame_num, width, height

def get_3d_img_for_one_case(img_path_list):
    img_3d=[]
    for idx, img_path in enumerate(img_path_list):
        print("progress: "+str(idx/len(img_path_list))+"; "+str(img_path), end="\r")
        img_slice, frame_num, _, _ = loadFile(img_path)
        assert frame_num==1
        ds = pydicom.read_file(img_path)
        if 'AXIAL' in ds[0x0008, 0x0008] and img_slice.shape == (1, 512, 512):
            img_3d.append(img_slice)
    img_3d=np.array(img_3d)
    return img_3d.reshape(img_3d.shape[0], img_3d.shape[2], img_3d.shape[3])

In [88]:
save_path = "./data/BE_CT_3D"
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [89]:
for key, val in data_dict.items():
    img = get_3d_img_for_one_case(val)
    sitk.WriteImage(sitk.GetImageFromArray(img), os.path.join(save_path, key+'.nii.gz'))

ValueError: too many values to unpack (expected 3)